In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
queries_df=pd.read_csv('/content/Queries.csv')

In [ ]:
queries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Top queries  1000 non-null   object 
 1   Clicks       1000 non-null   int64  
 2   Impressions  1000 non-null   int64  
 3   CTR          1000 non-null   object 
 4   Position     1000 non-null   float64
dtypes: float64(1), int64(2), object(2)
memory usage: 39.2+ KB


In [ ]:
queries_df.head()

,Top queries,Clicks,Impressions,CTR,Position
0,number guessing game python,5223,14578,35.83%,1.61
1,thecleverprogrammer,2809,3456,81.28%,1.02
2,python projects with source code,2077,73380,2.83%,5.94
3,classification report in machine learning,2012,4959,40.57%,1.28
4,the clever programmer,1931,2528,76.38%,1.09


In [ ]:
#cleaning ctr columns
queries_df['CTR']=queries_df['CTR'].replace({'%':''},regex=True).astype('float')/100

In [ ]:
queries_df.head()

,Top queries,Clicks,Impressions,CTR,Position
0,number guessing game python,5223,14578,0.3583,1.61
1,thecleverprogrammer,2809,3456,0.8128,1.02
2,python projects with source code,2077,73380,0.0283,5.94
3,classification report in machine learning,2012,4959,0.4057,1.28
4,the clever programmer,1931,2528,0.7638,1.09


In [ ]:
# clean and finding queries by using regular expression
import  re
def clean_and_split(query):
  words= re.findall(r'\b[a-zA-Z]+\b',query.lower())
  return words

In [ ]:
from collections import Counter
word_counts=Counter()
for query in queries_df['Top queries']:
  word_counts.update(clean_and_split(query))

In [ ]:
word_count_df=pd.DataFrame(word_counts.most_common(20),columns=['Word','Frequency'])

In [ ]:
import plotly.express as px
fig=px.bar(word_count_df,x='Word',y='Frequency')
fig.show()


In [ ]:
top_queries_by_clicks=queries_df.nlargest(10,'Clicks')[['Top queries','Clicks']]
top_queries_by_impression=queries_df.nlargest(10,'Impressions')[['Top queries','Impressions']]
fig_click=px.bar(top_queries_by_clicks,x='Top queries',y='Clicks')
fig_impressions=px.bar(top_queries_by_impression,x='Top queries',y='Impressions')
fig_click.show()
fig_impressions.show()

In [ ]:
# queries with highest and lowest ctr
top_ctr=queries_df.nlargest(10,'CTR')[['Top queries','CTR']]
bottom_ctr=queries_df.nsmallest(10,'CTR')[['Top queries','CTR']]
fig_top_ctr=px.bar(top_ctr,x='Top queries',y='CTR')
fig_bottom_ctr=px.bar(bottom_ctr,x='Top queries',y='CTR')
fig_top_ctr.show()
fig_bottom_ctr.show()

In [ ]:
correlation_matrix=queries_df.corr()
fig_corr=px.imshow(correlation_matrix,text_auto=True)
fig_corr.show()

<ipython-input-27-103d096cc815>:1: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



In [ ]:
queries_df.columns

Index(['Top queries', 'Clicks', 'Impressions', 'CTR', 'Position'], dtype='object')

Using Ensemble Forest algorithm for anamoly detection

In [ ]:
from sklearn.ensemble import IsolationForest
features=queries_df[['Clicks', 'Impressions', 'CTR', 'Position']]

In [ ]:
iso_forest=IsolationForest(n_estimators=100,contamination=0.01)
iso_forest.fit(features)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but IsolationForest was fitted with feature names



IsolationForest(contamination=0.01)

In [ ]:
queries_df['anamoly']=iso_forest.predict(features)

In [ ]:
anamolies=queries_df[queries_df['anamoly']==-1]

In [ ]:
print(anamolies[['Top queries', 'Clicks', 'Impressions', 'CTR', 'Position']])

                          Top queries  Clicks  Impressions     CTR  Position
0         number guessing game python    5223        14578  0.3583      1.61
1                 thecleverprogrammer    2809         3456  0.8128      1.02
2    python projects with source code    2077        73380  0.0283      5.94
4               the clever programmer    1931         2528  0.7638      1.09
15         rock paper scissors python    1111        35824  0.0310      7.19
21              classification report     933        39896  0.0234      7.53
34           machine learning roadmap     708        42715  0.0166      8.97
82                           r2 score     367        56322  0.0065      9.33
91     facebook programming languages     346        36055  0.0096      1.58
232                    standardscaler     177        39267  0.0045     10.23
